In [1]:
import json
import pandas as pd
from sklearn.externals import joblib
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.naive_bayes import GaussianNB
from string import punctuation
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from unidecode import unidecode
from ethnicity_features import Selector, WordCount, NameLength, FirstLast, DictFirstNameFeatures, VowelsShare
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [2]:
class EP:
    
    def __init__(self, ethnicity):  
        
        assert ethnicity in ('iranian polish italian german japanese vietnamese'
                             ' portuguese fijian indian croatian greek'
                             ' serbian spanish korean thai turkish' 
                             ' english cambodian russian').split(), f'incorrect ethnicity: {ethnicity}!'
        
        datafile = f'/Users/ik/Data/names/training-{ethnicity}.csv'
        self.target_col = f'is_{ethnicity}'
        
        self.data = pd.read_csv(datafile).sample(frac=1.)
        self.data['full_name'] = self.data['full_name'].apply(lambda _: unidecode(_))
        
        assert self.target_col in self.data.columns, f'there is no {self.target_col} in data file!'  
        
        try:
            self.NODOUBT_FIRST_NAMES = list({line.strip() for line in open(f'/Users/ik/Data/names/real_{ethnicity}.txt','r').readlines() if line.strip()})
        except:
            self.NODOUBT_FIRST_NAMES = []
        print(self.NODOUBT_FIRST_NAMES)
        
    def add_nodoubt_names(self):
        
        print(f'adding {len(self.NODOUBT_FIRST_NAMES)} first names..')
        
        self.data = pd.concat([self.data, pd.DataFrame({'full_name': self.NODOUBT_FIRST_NAMES, 
                                                        self.target_col: [1]*len(self.NODOUBT_FIRST_NAMES)})], 
                              ignore_index=True).sample(frac=1.)
        return self

In [3]:
# class Selector(BaseEstimator, TransformerMixin):
#     """
#     select a columns from a data frame and return as a list
#     """
#     def __init__(self, col_name):
#         self.col_name = col_name
    
#     def fit(self, x, y=None):
#         return self

#     def transform(self, x):
#         return '_start_' + x[self.col_name] + '_end_'

# class WordCount(BaseEstimator, TransformerMixin):
#     """
#     select a columns from a data frame and return as a list
#     """
#     def __init__(self):
#         pass
    
#     def fit(self, x, y=None):
#         return self

#     def transform(self, x):
#         res = x.apply(lambda _: len(_.split())).values.reshape(x.shape[0],1)
#         return res

# class NameLength(BaseEstimator, TransformerMixin):
#     """
#     return the length of the full name
#     """
#     def __init__(self):
#         pass
    
#     def fit(self, x, y=None):
#         return self

#     def transform(self, x):
#         res = x.str.len().values.reshape(x.shape[0],1)
#         return res

# class FirstLast(BaseEstimator, TransformerMixin):
#     """
#     is the first word longer than the last one
#     """
#     def __init__(self):
#         pass
    
#     def fit(self, x, y=None):
#         return self

#     def transform(self, x):
#         res = x.apply(lambda _: np.argmax([len(p) for i, p in enumerate(_.split()) if i in [0,len(_.split())-1]])).values.reshape(x.shape[0],1)
#         return res

In [4]:
if __name__ == '__main__':
    
    ETHN = 'fijian'
    
    ic = EP(ETHN).add_nodoubt_names()
    
    # split into the training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(ic.data.drop([ic.target_col], axis=1), 
                                                        ic.data[ic.target_col],  test_size=0.3, 
                                                        random_state=391, stratify=ic.data[ic.target_col])
    

    pipe = Pipeline([('select_fullname', Selector('full_name')),  # df with 1 column
                     ('features', FeatureUnion(
                         [('char_level', CountVectorizer(strip_accents='ascii', analyzer='char', 
                                                          ngram_range=(2, 4))),
                          ('word_level', CountVectorizer(strip_accents='ascii', analyzer='word',
                                                       ngram_range=(1,3))),
                          ('word_count', WordCount()),
                          ('dict_fname', DictFirstNameFeatures(ic.NODOUBT_FIRST_NAMES)),
                          ('vow_share', VowelsShare()),
                          ('full_name_length', NameLength()),
                          ('firstlast', FirstLast())],
                              transformer_weights={
                                            'char_level': 0.8,
                                            'word_level': 0.5,
                                            'word_lengths': 1.0,})),
                     ('normalise', Normalizer()),
#                      ('pca', TruncatedSVD(n_components=120)),
                     ('clf', VotingClassifier(estimators=[('sgd', SGDClassifier(max_iter=1000)), 
                                                           ('rf', RandomForestClassifier(class_weight='balanced')),
                                                            ('svc', SVC(C=0.0001)),
                                                             ('abst', AdaBoostClassifier(n_estimators=200)),
                                                             ('gb', GradientBoostingClassifier(n_estimators=200))], 
                                                                    voting='hard'))])
    
    param_grid = {
#         'features__transformer_weights': [{'char_level': 0.2,
#                                                      'word_level': 0.1,
#                                                      'word_count':0.4,
#                                                     'full_name_length': 0.5}, 
#                                                    {'char_level': 0.9,
#                                                      'word_level': 0.2,
#                                                      'word_lengths':0.9,
#                                                    'full_name_length': 0.3},
#                                                    {'char_level': 0.9,
#                                                      'word_level': 0.3,
#                                                      'word_count':0.2,
#                                                    'full_name_length': 0.7}],
                    'clf__sgd__loss': ['modified_huber'], # also 'hinge', 'perceptron', 'log'
                    'clf__sgd__penalty': ['l2'],  # 'elasticnet', 'l1'
                    'clf__rf__n_estimators': [100, 200],
                    'clf__rf__max_depth': [None, 3],
                    'clf__svc__C': [1e-6, 1e-5, 1e-4, 1e-2, 1.]}  # default is 1.
#                  'pca__n_components': [30, 120, 400]}
    
    grid_search = GridSearchCV(pipe, param_grid=param_grid, verbose=10)
    
    grid_search.fit(X_train, y_train)
    
    joblib.dump(grid_search.best_estimator_, f'{ETHN}_model.pkl', compress = 1)

['alexade', 'kalara', 'nimlesh', 'tarlae', 'lelea', 'walota', 'selai', 'apenisa', 'makereta', 'ifereimi', 'ilikimi', 'iloilo', 'asenaca', 'nete', 'joni', 'karalaini', 'miliana', 'keresi', 'yasi', 'josivini', 'elenoa', 'maciu', 'ilisapeci', 'elisapeci', 'kona', 'sovaia', 'iloilovatu', 'alitia', 'semesa', 'siteri', 'esiteri']
adding 31 first names..
Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-06 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.3s remaining:    0.0s


[CV]  clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-06, score=0.9106145251396648, total=   6.1s
[CV] clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-06 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   10.6s remaining:    0.0s


[CV]  clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-06, score=0.9134078212290503, total=   4.0s
[CV] clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-06 


...

[CV]  clf__rf__max_depth=3, clf__rf__n_estimators=200, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1.0, score=0.9189944134078212, total=   6.6s
[CV] clf__rf__max_depth=3, clf__rf__n_estimators=200, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1.0 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  clf__rf__max_depth=3, clf__rf__n_estimators=200, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1.0, score=0.8938547486033519, total=   4.6s
[CV] clf__rf__max_depth=3, clf__rf__n_estimators=200, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1.0 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  5.8min finished


[CV]  clf__rf__max_depth=3, clf__rf__n_estimators=200, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1.0, score=0.9129213483146067, total=   4.6s


In [5]:
print(classification_report(y_test, grid_search.predict(X_test)))

             precision    recall  f1-score   support

          0       0.91      0.98      0.94       304
          1       0.95      0.80      0.87       156

avg / total       0.92      0.92      0.92       460



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [6]:
test_data = pd.DataFrame({'full_name': ['han walota', 'xi','babak', 'igor souza', 'yash patel', 'david perron',
                                        'andrzej woods','otto floms', 'suzuki', 'iloilovatu', 
                                        'edilson', 'kawasaki','silva', 'james tran', 
                                        'bo nguyen', 'zlatan ibrahimovic', 'ronaldo barbosa', 'natalia klanchinkova', 
                                        'bob', 'keresi right','kalara',  'kona reed', 'andreas vlachos', 'con', 'nemanja vidic']})

test_data['prediction'] = grid_search.predict(test_data)
test_data['prediction'] = test_data['prediction'].apply(lambda _: 'yes' if _ else 'no')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [7]:
grid_search.best_params_

{'clf__rf__max_depth': None,
 'clf__rf__n_estimators': 200,
 'clf__sgd__loss': 'modified_huber',
 'clf__sgd__penalty': 'l2',
 'clf__svc__C': 1e-05}

In [8]:
for t in zip(X_test.values, grid_search.predict(X_test), y_test.values):
    if t[1] != t[2]:
        print(f'{t[0][0].upper()} predicted {t[1]} but it is {t[2]}')

NORMAN LIGAIRI predicted 0 but it is 1
JIKO LUVENI predicted 0 but it is 1
FRANK SAKETA predicted 0 but it is 1
JAMES AH KOY predicted 0 but it is 1
PITA RABO predicted 0 but it is 1
AKUILA UATE predicted 0 but it is 1
DIXON SEETO predicted 0 but it is 1
CAMPESE MA'AFU predicted 0 but it is 1
ELIAS ANTONIOU predicted 1 but it is 0
TAKUMI NAGURA predicted 1 but it is 0
RYOTA NAGAKI predicted 1 but it is 0
INIA BOKO predicted 0 but it is 1
CAROLINE PUAMAU predicted 0 but it is 1
ANDREW DEOKI predicted 0 but it is 1
SHREE RAMLU predicted 0 but it is 1
LORIMA DAU predicted 0 but it is 1
PRAMOD RAE predicted 0 but it is 1
JOJI BANUVE predicted 0 but it is 1
PAUL ELAISA predicted 1 but it is 0
SETAREKI HUGHES predicted 0 but it is 1
SAMRESAN PILLAY predicted 0 but it is 1
TUI KAMIKAMICA predicted 0 but it is 1
SULIANO TAWANAKORO predicted 0 but it is 1
SANGEETA NIRANJAN predicted 0 but it is 1
TULSI RAM KHELWAN predicted 0 but it is 1
LAVENIA PADARATH predicted 0 but it is 1
DAVID RODAN pred

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
grid_search.best_estimator_

Pipeline(memory=None,
     steps=[('select_fullname', Selector(col_name='full_name')), ('features', FeatureUnion(n_jobs=1,
       transformer_list=[('char_level', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True...      warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None))])